# Generate summaries for each restaurant

In [1]:
import spacy
nlp = spacy.load('en')
import nltk
from nltk.corpus import stopwords
import pandas as pd
import numpy as np
import pickle

import re

import nltk
import re
import string
from nltk.corpus import stopwords

nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = nltk.stem.WordNetLemmatizer()
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()

from nltk.corpus import stopwords

import gensim
import pyLDAvis
from pyLDAvis import gensim as gensimvis
import spacy

from gensim.models.coherencemodel import CoherenceModel
from gensim.models.ldamodel import LdaModel
from gensim.corpora.dictionary import Dictionary
from numpy import array

import logging
from tqdm import tqdm
from pprint import pprint

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import NMF
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer 

[nltk_data] Downloading package wordnet to /home/ubuntu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [15]:
df = pd.read_pickle('pizza_sentences_sample.pkl')

In [3]:
bus_df = pd.read_pickle('business.pkl')


In [4]:
df.head(1)

,index,review_index,sentence_index,sentence,review_id,cleaned,topic,topic_name,sentiment,sentiment_val,business_id,name
0,45,491,0,I recently visited Pieology for the first time...,CsZu4mX7zrVw5kfM6Ca9wQ,"[pieology, time, friend, selection, taste, pie]",4,variety/options,"(1, Negative)",1.0,WWWufR135GEXKynwYjLdGg,Pieology Pizzeria


**Sub-topics within each main topic**

**Groupby topic/subtopic with average sentiment value and frequency of topic/subtopic**

In [53]:
name = "Pieology Pizzeria"
business_id = df['business_id'][df['name']==name][0]
stars = bus_df['stars'][bus_df['business_id'] == 'business_id']
business_id

'WWWufR135GEXKynwYjLdGg'

In [61]:
stars = pd.Series(bus_df['stars'][bus_df['business_id'] == business_id]).iloc[0]
stars

4.0

In [6]:
counts = df.groupby(['business_id']).size().reset_index(name='counts')
aggs = pd.merge (df, counts , how = 'left', on='business_id')


aggs = aggs.groupby(['business_id','topic','topic_name']).agg({'sentiment_val':'mean', 
                                                'index':lambda x: x.nunique(),
                                                'counts': 'mean'                                                
                                               }).reset_index()
aggs.columns = ['business_id','topic','topic_name','avg_sentiment', 'topic_count', 'sentence_count']
aggs['importance'] = aggs['topic_count']/aggs['sentence_count']
aggs = aggs.sort_values(by = ['business_id', 'importance'], ascending = False)
aggs.head()

,business_id,topic,topic_name,avg_sentiment,topic_count,sentence_count,importance
1229,zmfa8RNxqSuOJ9Nx-CMs_Q,0,pizza quality and style,1.942529,87,347,0.250720
1233,zmfa8RNxqSuOJ9Nx-CMs_Q,4,variety/options,2.156627,83,347,0.239193
1230,zmfa8RNxqSuOJ9Nx-CMs_Q,1,wait times,2.075000,80,347,0.230548
1232,zmfa8RNxqSuOJ9Nx-CMs_Q,3,experience,1.786885,61,347,0.175793
1231,zmfa8RNxqSuOJ9Nx-CMs_Q,2,service,2.194444,36,347,0.103746


In [8]:
list(aggs['topic'][aggs['business_id'] == 'zmfa8RNxqSuOJ9Nx-CMs_Q'])

[0, 4, 1, 3, 2]

**Topic order of importance for each restaurant**  
*For each restaurant, get list of topic/subtopics by order of frequency*  
*Return infograph of average sentiment for each topic/subtopic*
  
**Representative sentence in each topic/subtopic**  
*For each restaurant topic/subtopic item, get representative sentence*    
*Append sentences to summary paragraph in order of importance*

In [17]:
nlp = spacy.load('en', disable=['parser', 'ner'])
def clean_sentences(sentences):
    sentences = [re.sub("\n|\r", "", s) for s in sentences]    
    sentences = [s.lower() for s in sentences] # lower case    
    sentences = [s.translate(str.maketrans('', '', string.punctuation)) for s in sentences] # remove punctuation
    def lemmatize_alphanumeric(sentence):
        words = [lemmatizer.lemmatize(word) for word in sentence.split() if word.isalpha()
                and nlp.vocab[word].is_stop == False]
        sentence = ' '.join(words)
        return(sentence)    
    sentences = [lemmatize_alphanumeric(s) for s in sentences] # lemmatize and remove non alpha-numeric characters
    return(sentences)

In [9]:
sentences = ['the crust is great', 'great authentic pizza', 'they have the best crust',
             'the sauce is great and the crust is so doughy']
mat = get_matrix(sentences)

for i in mat:
    print(i, i.mean())

[1.         0.40824829 0.5        0.70710678] 0.6538387679126025
[0.40824829 1.         0.         0.28867513] 0.424230856264669
[0.5        0.         1.         0.35355339] 0.4633883476483183
[0.70710678 0.28867513 0.35355339 1.        ] 0.5873338265936585


In [10]:
def get_matrix(sentences):
    from nltk.corpus import stopwords
    stopwords = set(stopwords.words("english"))
    cv = CountVectorizer(stop_words=stopwords)
    
    X_cv = cv.fit_transform(sentences)    
    countvector = pd.DataFrame(X_cv.toarray(), columns=cv.get_feature_names())    
    cosines = cosine_similarity(X_cv)
    
    return(cosines)

In [11]:
def get_top_sentences(cosines, n):
    avg_cosines = []
    for c in range(len(cosines)):
        avg_cosines.append(cosines[c].mean())
    
    top_sentence_dict = {}
    for i in sorted(range(len(avg_cosines)), key=lambda i: avg_cosines[i])[-n:]:
        top_sentence_dict[i] = avg_cosines[i]
    
    top_sentence_i = []
    for w in sorted(top_sentence_dict, key=top_sentence_dict.get, reverse=True):
        top_sentence_i.append(w)
    
    return(top_sentence_i)

In [8]:
business_id = 'zmfa8RNxqSuOJ9Nx-CMs_Q'
topic = 2
n = 2

sentences = list(df['sentence'][(df['business_id'] == business_id) & (df['topic'] == topic)])
text = clean_sentences(sentences)
cosines = get_matrix(text)

top_sentence_i = get_top_sentences(cosines, n)

rep_sentences = []
for i in top_sentence_i:
    rep_sentences.append(sentences[i].lstrip())
    
rep_sentences


['Staff are super friendly', 'Staff are super friendly as well!']

In [25]:
def get_rep_sentences(business_id, topic, n):

    sentences = list(df['sentence'][(df['business_id'] == business_id) & (df['topic'] == topic)])
    
    text = clean_sentences(sentences)
    
    cosines = get_matrix(text)
    
    top_sentence_i = get_top_sentences(cosines, n)
    
    rep_sentences = []
    for i in top_sentence_i:
        rep_sentences.append(sentences[i].lstrip())
    
    return(rep_sentences)


def get_rep_sentences_tname(business_id, topic_name, n):
    sentences = list(df['sentence'][(df['business_id'] == business_id) & (df['topic_name'] == topic_name)])    
    text = clean_sentences(sentences)    
    cosines = get_matrix(text)
    top_sentence_i = get_top_sentences(cosines, n)    
    rep_sentences = []
    for i in top_sentence_i:
        rep_sentences.append(sentences[i].lstrip())    
    return(rep_sentences)

In [13]:
def print_rep_sentences(business_id, topic, n):
    rep_sentences = '. '.join(get_rep_sentences(business_id, topic, n))
    return(rep_sentences)

In [19]:
df.topic.unique()

array([4, 0, 1, 3, 2])

In [18]:
get_rep_sentences('zmfa8RNxqSuOJ9Nx-CMs_Q', 2, 5)

['Staff are super friendly',
 'Staff are super friendly as well!',
 'Very attentive staff',
 'The attending staff was super attentive, and engaging, made sure our table always had food/condiments while we were waiting for the main course',
 'The wait staff are wonderful here']

In [29]:
print_rep_sentences('zmfa8RNxqSuOJ9Nx-CMs_Q', 2, 5)

'Staff are super friendly. Staff are super friendly as well!. Very attentive staff. The attending staff was super attentive, and engaging, made sure our table always had food/condiments while we were waiting for the main course. The wait staff are wonderful here'

In [28]:
get_rep_sentences_tname('zmfa8RNxqSuOJ9Nx-CMs_Q', 'pizza quality and style', 5)

['The bread and tomato sauce was a novel idea and the bruschetta was also novel- on crispy pizza dough with fresh tomatoes',
 '0/10: The bread and tomato sauce appetizer everyone raves about',
 'Oh, and their bread and especially tomato dipping sauce were delicious',
 '50 for Bruschetta (tomatoes and bread - really?)',
 'The pizza was horrible, hardly a toasted bread with tomato sauce and a few small pieces of cheese']

In [22]:
def get_aggs(business_id, df):
    
    bus = df[df['business_id'] == business_id]
    
    sentence_count = bus.groupby(['business_id']).size().reset_index(name='counts')
    
    bus = pd.merge (bus, sentence_count , how = 'left', on='business_id')
    aggs = bus.groupby(['business_id','topic','topic_name']).agg({'sentiment_val':'mean', 
                                                 'index':lambda x: x.nunique(),
                                                 'counts': 'mean'                                                
                                                }).reset_index()
    aggs.columns = ['business_id','topic','topic_name','avg_sentiment', 'topic_count', 'sentence_count']
    aggs['importance'] = aggs['topic_count']/aggs['sentence_count']
    aggs = aggs.sort_values(by = ['business_id', 'importance'], ascending = False)
    
    return(aggs)

In [20]:
def get_aspect_aggs(name, df):
    business_id = df['business_id'][df['name']==name][0]
    aggs = get_aggs(business_id, df)
    aggs['sentiment_score'] = [(100*X/3.0) for X in aggs['avg_sentiment']]
    aggs = pd.Series(aggs.sentiment_score.values,index=aggs.topic_name).to_dict()
    return(aggs)

In [43]:
def normalize(df, cols):    
    for col in cols:
        max_value = df[col].max()
        min_value = df[col].min()
        df[col] = [(X - min_value) / (max_value - min_value) for X in df[col]]
    return (df)
df = normalize(df, ['sentiment_val'])
df.describe()

,index,review_index,sentence_index,topic,sentiment_val
count,94203.000000,9.420300e+04,94203.000000,94203.000000,94062.000000
mean,293870.850865,2.711284e+06,6.066505,1.520228,0.480197
std,199422.771554,1.806617e+06,7.441491,1.539223,0.220223
min,45.000000,4.910000e+02,0.000000,0.000000,0.000000
25%,112426.500000,1.059243e+06,1.000000,0.000000,0.250000
50%,257604.000000,2.349673e+06,4.000000,1.000000,0.500000
75%,464263.500000,4.271677e+06,8.000000,3.000000,0.750000
max,728216.000000,6.682752e+06,89.000000,4.000000,1.000000


In [33]:
aggs = get_aspect_aggs("Pieology Pizzeria", df)
aggs

{'pizza quality and style': 70.3499079189687,
 'wait times': 62.12121212121212,
 'variety/options': 74.58333333333333,
 'service': 65.59139784946237,
 'experience': 73.01587301587303}

In [41]:
business_id = 'zmfa8RNxqSuOJ9Nx-CMs_Q'
results_all = []
for k, v in aggs.items():
    results_dict = {}
    results_dict['aspect'] = k
    results_dict['score'] = v
    results_dict['sentences'] = get_rep_sentences_tname(business_id, k, 3)
    results_all.append(results_dict)
results_all

[{'aspect': 'pizza quality and style',
  'score': 70.3499079189687,
  'sentences': ['The bread and tomato sauce was a novel idea and the bruschetta was also novel- on crispy pizza dough with fresh tomatoes',
   '0/10: The bread and tomato sauce appetizer everyone raves about',
   'Oh, and their bread and especially tomato dipping sauce were delicious']},
 {'aspect': 'wait times',
  'score': 62.12121212121212,
  'sentences': ['This was my first time going to Positano, it was a Sunday evening so the restaurant was not that busy',
   'Have been here 7 times',
   'We love this restaurant, the staff, and food are great']},
 {'aspect': 'variety/options',
  'score': 74.58333333333333,
  'sentences': ['Service is good',
   '4/10: Service',
   'Great food, service and value']},
 {'aspect': 'service',
  'score': 65.59139784946237,
  'sentences': ['Staff are super friendly',
   'Staff are super friendly as well!',
   'Very attentive staff']},
 {'aspect': 'experience',
  'score': 73.01587301587303

In [20]:
list(df['business_id'][df['name']=='La Famiglia'])[0]

'TPvVGmT4AY1mloktV-2Dpw'

In [21]:
def get_summary(name, df):
    business_id = list(df['business_id'][df['name']==name])[0]
    
    aggs = get_aggs(business_id, df)
    
    summary = []
    for t in list(aggs['topic'][aggs['business_id'] == business_id]):
        summary.append(get_rep_sentences(business_id, t, 1)[0])
    summary = '. '.join(summary)
    return(summary)

In [22]:
get_summary("La Famiglia", df)

'This pizza place makes my list of top five pizza places in Arizona very good pizza. Every time we order a take out pizza they give us a pick up time and it is NEVER ready on time, even when the restaurant is not busy at all. Great food and service. Food. I had a pizza and my husband the chicken Marsala'

**Aspect overview**

In [39]:
df['sentiment_val'].describe()

count    94062.000000
mean         1.920786
std          0.880893
min          0.000000
25%          1.000000
50%          2.000000
75%          3.000000
max          4.000000
Name: sentiment_val, dtype: float64

In [6]:
# def get_topics_lda(tokenized_text, num_topics):
#     id2word = gensim.corpora.Dictionary(tokenized_text)
#     corpus = [id2word.doc2bow(t) for t in tokenized_text]
#     lda_model = gensim.models.ldamulticore.LdaMulticore(corpus=corpus,
#                                            id2word=id2word,
#                                            num_topics=num_topics, 
#                                            random_state=100,
#                                            chunksize=100,
#                                            passes=128,
#                                            per_word_topics=True)
#     return(lda_model)

In [5]:
# df_subs = pd.DataFrame()
# for t in df['topic']:
#     print('topic:' t)
    
#     df_tmp = df['cleaned'][df['topic'] == t]
    
#     lda_model = get_topics_lda(df_tmp, 3)
    
#     lda_model.print_topics(num_words=20)
    
#     pd.to_pickle(lda_model, 'lda_subtopics_20k_'+str(t)+'.pkl')
    
#     get_document_topics = [lda_model.get_document_topics(item) for item in corpus]
#     topics_dict = {}
#     for i, doc in enumerate(get_document_topics):
#         topics_dict[i] = max(doc,key=itemgetter(1))[0]
#     main_topic = pd.DataFrame(list(topics_dict.items()), columns = ['index','sub_topic'+str(t)])
    
#     df_tmp = pd.concat([df_tmp.reset_index(), main_topic['sub_topic'+str(t)]], axis=1)
    
#     df_subs = pd.concat[df_subs, df_tmp]
    
#     del(df_tmp)

#     print('completed')

0    27032
3    15616
1    14263
4    14048
2    10900
Name: topic, dtype: int64